In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import current_timestamp, col, count, countDistinct, sum, avg
from pyspark.sql.types import StructField, StructType, StringType, IntegerType, DoubleType
from delta.tables import *
from delta import *
from delta import configure_spark_with_delta_pip


In [3]:
builder = SparkSession \
        .builder \
        .appName('healthcare_Aggregations')\
        .master('local')\
        .config("spark.driver.memory","2g")\
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")\
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")\
        .config("spark.sql.execution.arrow.pyspark.enabled", "true")


spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [4]:
spark

In [ ]:
silver = "silver"

In [9]:
source = spark.read \
                .format("delta").load(f"{silver_folder}\\healthcare_trans")
                
                


In [10]:
source.count()

26294

In [11]:
source.printSchema()

root
 |-- PATIENTS_ID: string (nullable = true)
 |-- ENCOUNTERS_ID: string (nullable = true)
 |-- FIRST: string (nullable = true)
 |-- LAST: string (nullable = true)
 |-- GENDER: string (nullable = true)
 |-- CITY: string (nullable = true)
 |-- BIRTHDATE: date (nullable = true)
 |-- DEATHDATE: date (nullable = true)
 |-- ENCOUNTERCLASS: string (nullable = true)
 |-- REASONCODE: long (nullable = true)
 |-- REASONDESCRIPTION: string (nullable = true)
 |-- BASE_COST: integer (nullable = true)
 |-- BASE_ENCOUNTER_COST: double (nullable = true)
 |-- TOTAL_CLAIM_COST: double (nullable = true)
 |-- PAYER: string (nullable = true)
 |-- PAYER_COVERAGE: double (nullable = true)
 |-- PAYER_NAME: string (nullable = true)
 |-- INGESTION_DATE: timestamp (nullable = true)
 |-- MODIFICATION_DATE: timestamp (nullable = true)
 |-- SOURCE: string (nullable = true)



In [9]:
source.show()

+--------------------+--------------------+-----------+-------------+------+----------+----------+----------+----+----------+--------------------+---------+-------------------+----------------+--------------------+--------------+--------------+--------------------+
|         patients_id|       encounters_id|      FIRST|         LAST|GENDER|      CITY| BIRTHDATE| DEATHDATE|NAME|REASONCODE|   REASONDESCRIPTION|BASE_COST|BASE_ENCOUNTER_COST|TOTAL_CLAIM_COST|               PAYER|PAYER_COVERAGE|ENCOUNTERCLASS|          PAYER_NAME|
+--------------------+--------------------+-----------+-------------+------+----------+----------+----------+----+----------+--------------------+---------+-------------------+----------------+--------------------+--------------+--------------+--------------------+
|5e129b38-3d63-369...|0002c38a-54e9-078...|  Jolene291|     Kling921|     F|    Boston|1961-08-06|      NULL|NULL|  68496003|      Polyp of colon|     3737|              85.55|        10224.23|6e2f1a2d-

In [13]:
source.createOrReplaceTempView("health")

In [ ]:
sqlDF = spark.sql("SELECT * FROM \
                  health LIMIT 3")
sqlDF.show()

+--------------------+--------------------+-----------+-----------+------+----------+----------+----------+--------------+----------+--------------------+---------+-------------------+----------------+--------------------+--------------+--------------------+--------------------+--------------------+------+
|         PATIENTS_ID|       ENCOUNTERS_ID|      FIRST|       LAST|GENDER|      CITY| BIRTHDATE| DEATHDATE|ENCOUNTERCLASS|REASONCODE|   REASONDESCRIPTION|BASE_COST|BASE_ENCOUNTER_COST|TOTAL_CLAIM_COST|               PAYER|PAYER_COVERAGE|          PAYER_NAME|      INGESTION_DATE|   MODIFICATION_DATE|SOURCE|
+--------------------+--------------------+-----------+-----------+------+----------+----------+----------+--------------+----------+--------------------+---------+-------------------+----------------+--------------------+--------------+--------------------+--------------------+--------------------+------+
|5e129b38-3d63-369...|0002c38a-54e9-078...|  Jolene291|   Kling921|     F|  

In [17]:
#Health Analysis
pop_health = spark.sql("SELECT CITY, ENCOUNTERCLASS, REASONCODE, REASONDESCRIPTION \
                       FROM health LIMIT 3")
pop_health.show()

+-------+--------------+----------+-----------------+
|   CITY|ENCOUNTERCLASS|REASONCODE|REASONDESCRIPTION|
+-------+--------------+----------+-----------------+
| Boston|    ambulatory|  68496003|   Polyp of colon|
|Chelsea|    outpatient|  68496003|   Polyp of colon|
|   Hull|    outpatient|      NULL|             NULL|
+-------+--------------+----------+-----------------+



In [15]:
spark.stop

<bound method SparkSession.stop of <pyspark.sql.session.SparkSession object at 0x0000018943061C90>>